In [1]:
import igraph as ig#version 0.10.4 returned Node with 17916 
import cairo
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df_all = pd.read_csv('model_data.csv')
df_dist = pd.read_csv('model_data_dist.csv')

In [3]:
df_inp = pd.read_csv('Train_Inpatientdata-1542865627584.csv')
df_inp['InOut'] = 'In'
df_outp = pd.read_csv('Train_Outpatientdata-1542865627584.csv')
df_outp['InOut'] = 'Out'
df_label = pd.read_csv('Train-1542865627584.csv')

In [4]:
df = pd.concat([df_inp, df_outp], ignore_index=True)
df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,InOut
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,7092.000,NaN,NaN,NaN,NaN,NaN,In
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,40390,4019,NaN,331.000,NaN,NaN,NaN,NaN,NaN,In
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,29620,20300,NaN,3893.000,NaN,NaN,NaN,NaN,NaN,In


In [5]:
def if_na(column_value):
    if column_value.isna():
        return 0
    return 1

In [6]:
df['NumberOfPhysician'] = df['AttendingPhysician'].notna().astype(int) + df['OperatingPhysician'].notna().astype(int) + df['OtherPhysician'].notna().astype(int)
df['ClmDiagnosisCode_Count'] = (df['ClmDiagnosisCode_1'].notna().astype(int) + df['ClmDiagnosisCode_2'].notna().astype(int)
             + df['ClmDiagnosisCode_3'].notna().astype(int) + df['ClmDiagnosisCode_4'].notna().astype(int)
             + df['ClmDiagnosisCode_5'].notna().astype(int) + df['ClmDiagnosisCode_6'].notna().astype(int)
             + df['ClmDiagnosisCode_7'].notna().astype(int) + df['ClmDiagnosisCode_8'].notna().astype(int)
             + df['ClmDiagnosisCode_9'].notna().astype(int) + df['ClmDiagnosisCode_10'].notna().astype(int))
df['ClmProcedureCode_Count'] = (df['ClmProcedureCode_1'].notna().astype(int) + df['ClmProcedureCode_2'].notna().astype(int)
             + df['ClmProcedureCode_3'].notna().astype(int) + df['ClmProcedureCode_4'].notna().astype(int)
             + df['ClmProcedureCode_5'].notna().astype(int) + df['ClmProcedureCode_6'].notna().astype(int))
df = df[['BeneID','Provider','InscClaimAmtReimbursed','AttendingPhysician','NumberOfPhysician','ClmDiagnosisCode_Count','ClmProcedureCode_Count','InOut']]
df.head()

,BeneID,Provider,InscClaimAmtReimbursed,AttendingPhysician,NumberOfPhysician,ClmDiagnosisCode_Count,ClmProcedureCode_Count,InOut
0,BENE11001,PRV55912,26000,PHY390922,1,9,0,In
1,BENE11001,PRV55907,5000,PHY318495,2,3,1,In
2,BENE11001,PRV56046,5000,PHY372395,2,6,0,In
3,BENE11011,PRV52405,5000,PHY369659,3,9,1,In
4,BENE11014,PRV56614,10000,PHY379376,2,9,1,In


In [7]:
df_label = df_label.merge(df.loc[df['InOut'] == 'In'].groupby('Provider').count().reset_index()[['Provider','BeneID']].rename(columns={"BeneID": "In_BeneCount"}),on='Provider',how='left')

In [8]:
df_label = df_label.merge(df.loc[df['InOut'] == 'In'].groupby('Provider').mean().reset_index().rename(columns={"InscClaimAmtReimbursed": "In_InscClaimAmtReimbursed","NumberOfPhysician": "In_NumberOfPhysician","ClmDiagnosisCode_Count": "In_ClmDiagnosisCode_Count","ClmProcedureCode_Count": "In_ClmProcedureCode_Count"}),on='Provider',how='left')

In [9]:
df_label = df_label.merge(df.loc[df['InOut'] == 'Out'].groupby('Provider').count().reset_index()[['Provider','BeneID']].rename(columns={"BeneID": "Out_BeneCount"}),on='Provider',how='left')

In [10]:
df_label = df_label.merge(df.loc[df['InOut'] == 'Out'].groupby('Provider').mean().reset_index().rename(columns={"InscClaimAmtReimbursed": "Out_InscClaimAmtReimbursed","NumberOfPhysician": "Out_NumberOfPhysician","ClmDiagnosisCode_Count": "Out_ClmDiagnosisCode_Count","ClmProcedureCode_Count": "Out_ClmProcedureCode_Count"}),on='Provider',how='left')

In [11]:
df_label = df_label.merge(df.groupby('Provider').count().reset_index()[['Provider','BeneID']].rename(columns={"BeneID": "Full_BeneCount"}),on='Provider',how='left')

In [12]:
df_label = df_label.merge(df.groupby('Provider').mean().reset_index().rename(columns={"InscClaimAmtReimbursed": "Full_InscClaimAmtReimbursed","NumberOfPhysician": "Full_NumberOfPhysician","ClmDiagnosisCode_Count": "Full_ClmDiagnosisCode_Count","ClmProcedureCode_Count": "Full_ClmProcedureCode_Count"}),on='Provider',how='left')

In [13]:
df_label

,Provider,PotentialFraud,In_BeneCount,In_InscClaimAmtReimbursed,In_NumberOfPhysician,In_ClmDiagnosisCode_Count,In_ClmProcedureCode_Count,Out_BeneCount,Out_InscClaimAmtReimbursed,Out_NumberOfPhysician,Out_ClmDiagnosisCode_Count,Out_ClmProcedureCode_Count,Full_BeneCount,Full_InscClaimAmtReimbursed,Full_NumberOfPhysician,Full_ClmDiagnosisCode_Count,Full_ClmProcedureCode_Count
0,PRV51001,No,5.000,19400.000,1.600,7.200,0.600,20.000,382.000,1.600,2.200,0.000,25,4185.600,1.600,3.200,0.120
1,PRV51003,Yes,62.000,9241.935,1.645,8.113,0.774,70.000,466.714,1.429,2.714,0.000,132,4588.409,1.530,5.250,0.364
2,PRV51004,No,NaN,NaN,NaN,NaN,NaN,149.000,350.134,1.604,2.584,0.000,149,350.134,1.604,2.584,0.000
3,PRV51005,Yes,NaN,NaN,NaN,NaN,NaN,1165.000,241.124,1.599,2.589,0.000,1165,241.124,1.599,2.589,0.000
4,PRV51007,No,3.000,6333.333,1.333,7.333,0.333,69.000,213.188,1.536,2.797,0.000,72,468.194,1.528,2.986,0.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,NaN,NaN,NaN,NaN,NaN,28.000,380.000,1.464,2.071,0.000,28,380.000,1.464,2.071,0.000
5406,PRV57760,No,NaN,NaN,NaN,NaN,NaN,22.000,216.818,1.682,2.364,0.000,22,216.818,1.682,2.364,0.000
5407,PRV57761,No,NaN,NaN,NaN,NaN,NaN,82.000,225.244,1.610,2.671,0.000,82,225.244,1.610,2.671,0.000
5408,PRV57762,No,NaN,NaN,NaN,NaN,NaN,1.000,1900.000,1.000,1.000,0.000,1,1900.000,1.000,1.000,0.000


In [14]:
df_info = df_all
df_info = df_info.merge(df_dist.drop(['PotentialFraud'], axis=1))
df_info = df_info.merge(df_label.drop(['PotentialFraud'], axis=1))
df_info.to_csv('Model_Dataset.csv')